In [299]:
'''
Marshall Ma
maxiaoduke@gmail.com
May 2018

You can mark jobs as "interested" and/or "applied"
If you run it for the first time, it will save results as xlsx in the same folder of this notebook; 
If you run it repeatedly (and youan change search condition), new results will be added to previous results
(without dupliacting or wrtting off the interested/applied)

'''

'\nMarshall Ma\nmaxiaoduke@gmail.com\nMay 2018\n\nYou can mark jobs as "interested" and/or "applied"\nIf you run it for the first time, it will save results as xlsx in the same folder of this notebook; \nIf you run it repeatedly (and youan change search condition), new results will be added to previous results\n(without dupliacting or wrtting off the interested/applied)\n\n'

In [300]:
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import time

In [301]:
#https://www.indeed.com/jobs?as_and=&as_phr=&as_any=&as_not=&as_ttl=data+analyst&as_cmp=&jt=fulltime&st=&salary=&radius=50&l=United+States&fromage=any&limit=50&sort=&psf=advsrch
jobtitle = "intern".replace(" ","+")  #must replace spaces with +
jobtype = "internship"   # fulltime or internship
loc = "NC".replace("," , "%2C").replace(" ","+") #location format can be like "United States" "CA" "San Francisco, CA" 
pages = 1  # scrape 50*pages rows

df = pd.DataFrame(columns = ['interested', 'applied','title', 'company', 'city', 'state', 'link','description','search_date','job_id'])
count = 0
for pageN in range(0,pages):
    url = "https://www.indeed.com/jobs?as_and=&as_phr=&as_any=&as_not=&as_ttl="+jobtitle+"&as_cmp=&jt="+jobtype+"&st=&salary=&radius=50&l="+loc+"&fromage=any&limit=50&sort=&psf=advsrch"+"&start="+str(50*pageN)
    pagehtml = BeautifulSoup(requests.get(url).text, "html.parser")
    jobs = pagehtml.find_all(name='div', attrs={'class': 'row'})   # question: why not "row result clickcard"?
    count += len(jobs)
    time.sleep(1) # Allow 1 sec for page grabs
    
    for job in jobs:

        # get title
        try:
            title = job.find(name='a', attrs={'class':'turnstileLink'}).attrs['title']
        except:
            title = 'NotFound'

        # get company
        try:
            company = job.find(name='span', attrs={'class':'company'}).text.replace("\n","")
            while company[0]==" ":
                company = company[1:] 
        except:
            company = 'NotFound'

        # get location
        try:
            location = job.find(name='span', attrs={'class':'location'}).text
            if(", " in location):
                city = location.split(", ")[0]
                state = location.split(", ")[1][:2]
            else:
                city = '?'
                state = location
        except:
            city = 'NotFound'
            state = 'NotFound'

        #get link
        try:
            link = "https://www.indeed.com/viewjob?jk=" + job['data-jk']
        except:
            link = 'NotFound'

        #get job id
        try:
            job_id = job['data-jk']
        except:
            job_id = 'NotFound'

        #get search date
        today = datetime.date.today()
        
    
        #add to dataframe
        df = df.append({'title':title, 'company':company, 'city':city, 'state':state, 'link':link,
                        'search_date':today,'job_id':job_id}, ignore_index=True)

# clean duplicates        
df = df.drop_duplicates('job_id')

#get description
def get_description(link):
    if(link!='NotFound'):
        jobpage = BeautifulSoup(requests.get(link).text, "html.parser", from_encoding="utf-8")
        spans = jobpage.find_all('span', attrs={'class': 'summary'})
        description = ''
        for span in spans:
            description = description+span.text.strip()
        return(description)
    else:
        return('NotFound')

df['description'] = df['link'].apply(lambda x: get_description(x))

print(count,"records found;  ", len(df),"unique jobs scraped")

# take a look at the first 5 new results
df.head()




C:\Users\Marshall\Anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


55 records found;   55 unique jobs scraped


,interested,applied,title,company,city,state,link,description,search_date,job_id
0,NaN,NaN,Summer Intern/Summer Camp,Sterling Academy,Raleigh,NC,https://www.indeed.com/viewjob?jk=5d031135b3fc...,Summer Camp Interns Needed. Sterling Academy S...,2018-05-03,5d031135b3fc79eb
1,NaN,NaN,Industrial IT Intern,Biomérieux,Durham,NC,https://www.indeed.com/viewjob?jk=615e3bb3fcc8...,A strong Scientific and technological pioneeri...,2018-05-03,615e3bb3fcc80c51
2,NaN,NaN,Digital Marketing Intern,Cognetik,Cary,NC,https://www.indeed.com/viewjob?jk=b4e5c441dd6d...,"Digital MarketerLocation: Cary, North Carolina...",2018-05-03,b4e5c441dd6dcd27
3,NaN,NaN,Carolina CAT - Rental Operations Intern,Carolina CAT - Construction,Charlotte,NC,https://www.indeed.com/viewjob?jk=3e475cadad96...,We are looking for an eager Rental Operations ...,2018-05-03,3e475cadad965966
4,NaN,NaN,Carolina CAT - Sales and Marketing Intern,Carolina CAT - Construction,Charlotte,NC,https://www.indeed.com/viewjob?jk=e8383cc13da9...,The Sales and Marketing Intern will directly a...,2018-05-03,e8383cc13da9e77d


In [302]:
#merge results with previous ones if you have downloaded one on your computer (in the same folder as this ipynb file)
try:
    origin_df = pd.read_excel('jobs.xlsx')
    df = df.append(origin_df)
    df = df.drop_duplicates('job_id', keep='last')
    print(len(df)-len(origin_df),"new jobs found")
except:
    pass

# have a glance at results, old and new
print("You Currently have" , len(df), "records, old and new")
df.head()

3 new jobs found
You Currently have 58 records


,interested,applied,title,company,city,state,link,description,search_date,job_id
7,NaN,NaN,Data Scientist Intern,IBM,Research Triangle Park,NC,https://www.indeed.com/viewjob?jk=86a68a6c5268...,Job Description\nData Scientist is a role for ...,2018-05-03,86a68a6c526863ef
14,NaN,NaN,Marketing Intern,Progress Fitness Unlimited,Greensboro,NC,https://www.indeed.com/viewjob?jk=60964d89421f...,We are looking for an enthusiastic marketing i...,2018-05-03,60964d89421ffca2
54,NaN,NaN,Clinical Pharmacy Intern,"Physicians Pharmacy Alliance, Inc.",Cary,NC,https://www.indeed.com/viewjob?jk=d2044d453f8f...,"Work Site Location(s): Cary, NCApplicants are ...",2018-05-03,d2044d453f8f1adb
0,NaN,NaN,Industrial IT Intern,Biomérieux,Durham,NC,https://www.indeed.com/viewjob?jk=615e3bb3fcc8...,A strong Scientific and technological pioneeri...,2018-05-03 00:00:00,615e3bb3fcc80c51
1,NaN,NaN,Summer Intern/Summer Camp,Sterling Academy,Raleigh,NC,https://www.indeed.com/viewjob?jk=5d031135b3fc...,Summer Camp Interns Needed. Sterling Academy S...,2018-05-03 00:00:00,5d031135b3fc79eb


In [303]:
# save as Excel to local
df.to_excel('jobs.xlsx', index=False)

In [304]:
# clear records
df = df[0:0]